# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [2]:
import pandas as pd
import requests

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the ULR using the `requests` library.

In [3]:
# YOUR CODE HERE

URL = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(URL, headers=HEADERS)

Use Beautiful Soup to parse this string into a tree called `soup`

In [4]:
# YOUR CODE HERE
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [13]:
len(soup.find_all("table"))

10

We can use attributes like `class=` and `style=` to narrow down the list.

In [27]:
tables = soup.find_all("table", class_="wikitable")
print(len(tables))


7


At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [29]:
table = tables[1]
table

<table class="sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center" style="text-align:right">
<tbody><tr>
<th rowspan="2">City
</th>
<th rowspan="2"><abbr title="State or district">ST</abbr>
</th>
<th rowspan="2">2024<br/>estimate
</th>
<th rowspan="2">2020<br/>census
</th>
<th rowspan="2">Change
</th>
<th colspan="2">2020 land area
</th>
<th colspan="2">2020 density
</th>
<th rowspan="2">Location
</th></tr>
<tr class="static-row-header">
<th>mi<sup>2</sup>
</th>
<th>km<sup>2</sup>
</th>
<th>/ mi<sup>2</sup>
</th>
<th>/ km<sup>2</sup>
</th></tr>
<tr>
<td style="background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York City">New York</a></i><sup class="reference" id="cite_ref-5"><a href="#cite_note-5"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</td>
<td><a href="/wiki/New_York_(state)" title="New York (state)">NY</a>
</td>
<td style="text-align:right;">8,478,072
</td>
<td style="text-align:right;

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [ ]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [41]:
tbody = table.find("tbody")
tbody.find_all("tr")

rows = []
for city in tbody.find_all("tr"):
  cells = city.find_all("td")
  city_cell = cells[0] if len(cells) > 1 else None
  if not city_cell:
        continue

  a = city_cell.find("a", attrs={"title": True})
  city = a["title"].strip() if a else city_cell.get_text(strip=True)


  state_cell = cells[1] if len(cells) > 1 else None
  if not state_cell:
        continue
  a = state_cell.find("a", attrs={"title": True})
  state = a["title"].strip() if a else state_cell.get_text(strip=True)


  pop_text = cells[2].get_text(strip=True)
  population = int(pop_text.replace(",", ""))

  sqmi_text = cells[5].get_text(strip=True)
  squaremile = float(sqmi_text.replace(",", ""))


  rows.append({
      "city": city,
      "state": state,
      "population": population,
      "area": squaremile
               })




df_cities = pd.DataFrame(rows)
df_cities.head(30)
df_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        346 non-null    object 
 1   state       346 non-null    object 
 2   population  346 non-null    int64  
 3   area        346 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 10.9+ KB


Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Aside: Scraping an HTML table with Pandas



The Pandas command `read_html` can be used to scrape information from an HTML table on a webpage.

We can call `read_html` on the URL.

In [42]:
pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")

HTTPError: HTTP Error 403: Forbidden

However, this scrapes all the tables on the webpage, not just the one we want. As with Beautiful Soup, we can narrow the search by specifying the table attributes.

In [45]:
pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})

HTTPError: HTTP Error 403: Forbidden

This still returns 3 tables. As we remarked above, the table that we want is the first one (see `[0]` below).

In [44]:
df_cities2 = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})[0]
df_cities2

HTTPError: HTTP Error 403: Forbidden

Wait, that seemed much easier than using Beautiful Soup, and it returned a data frame, and we even got for free some formatting like removing the commas from the population! Why didn't we just use `read_html` in the first place? It's true the `read_html` works well when scraping information from an HTML *table*. Unfortunately, you often want to scrape information from a webpage that isn't conveniently stored in an HTML table, in which case `read_html` won't work. (It only searches for `<table>`, `<th>`, `<tr>`, and `<td>` tags, but there are many other HTML tags.) Though Beautiful Soup is not as simple as `read_html`, it is more flexible and thus more widely applicable.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [51]:
response = pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")
response

[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [54]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [53]:
from bs4 import BeautifulSoup
import re
import pandas as pd

URL = "http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# ---- get the response ----
response = requests.get(URL, headers=HEADERS)

soup = BeautifulSoup(response.content, "html.parser")

rows = []
for block in soup.find_all("div", class_="courseblock"):

    title_p = block.find("p", class_="courseblocktitle")
    if not title_p:
        continue

    # drop the "4 units" span so it doesn't end up in the title
    units = title_p.find("span", class_="courseblockhours")
    if units:
        units.extract()


    strong = title_p.find("strong")
    course = strong.get_text(" ", strip=True) if strong else title_p.get_text(" ", strip=True)


    desc_container = block.find("div", class_="courseblockdesc") or block.find("div", class_="noindent courseextendedwrap")
    terms = None
    if desc_container:
        text = desc_container.get_text(" ", strip=True)
        m = re.search(r"Terms?\s+typically\s+offered:\s*(.+?)(?:\.|$)", text, flags=re.I)
        if m:
            terms = m.group(1).strip()

    rows.append({"course": course, "terms": terms})

df = pd.DataFrame(rows)
df.head()



,course,terms
0,DATA 100. Data Science for All I.,None
1,DATA 301. Introduction to Data Science.,None
2,DATA 401. Data Science Process and Ethics.,None
3,DATA 402. Mathematical Foundations of Data Sci...,None
4,DATA 403. Data Science Projects Laboratory.,None


Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag